In [2]:
from SimulatedNetwork import SimulatedNetwork
from State.StaticStateSimple import StaticStateSimple
from Reward.LinearReward import LinearReward

ModuleNotFoundError: No module named 'SimulatedNetwork'

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

import sys
sys.path.append('/project/ai4s-hackathon/ai-sci-hackathon-2025/')

from rl_and_biological_network_project.SimulatedNetwork import SimulatedNetwork
from rl_and_biological_network_project.Code.State.StaticStateSimple import StaticStateSimple
from rl_and_biological_network_project.Code.Reward.LinearReward import LinearReward

class DQN(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 128)
        self.out = nn.Linear(128, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.out(x)

ModuleNotFoundError: No module named 'torch'

In [ ]:
import random
import numpy as np
import torch
from collections import deque

class DQNAgent:
    def __init__(self, state_dim, action_dim, lr=1e-4, gamma=0.99, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.995):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.gamma = gamma
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.q_net = DQN(state_dim, action_dim).to(self.device)
        self.target_net = DQN(state_dim, action_dim).to(self.device)
        self.target_net.load_state_dict(self.q_net.state_dict())

        self.optimizer = torch.optim.Adam(self.q_net.parameters(), lr=lr)
        self.memory = deque(maxlen=10000)
        self.batch_size = 64
        self.update_freq = 50
        self.learn_step = 0

    def select_action(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.randint(self.action_dim)
        state = torch.FloatTensor(state).unsqueeze(0).to(self.device)
        with torch.no_grad():
            q_values = self.q_net(state)
        return torch.argmax(q_values).item()

    def store(self, s, a, r, s_, done):
        self.memory.append((s, a, r, s_, done))

    def learn(self):
        if len(self.memory) < self.batch_size:
            return

        batch = random.sample(self.memory, self.batch_size)
        s, a, r, s_, d = zip(*batch)

        s = torch.FloatTensor(s).to(self.device)
        a = torch.LongTensor(a).unsqueeze(1).to(self.device)
        r = torch.FloatTensor(r).unsqueeze(1).to(self.device)
        s_ = torch.FloatTensor(s_).to(self.device)
        d = torch.FloatTensor(d).unsqueeze(1).to(self.device)

        q_eval = self.q_net(s).gather(1, a)
        with torch.no_grad():
            q_next = self.target_net(s_).max(1)[0].unsqueeze(1)
            q_target = r + self.gamma * q_next * (1 - d)

        loss = nn.MSELoss()(q_eval, q_target)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        self.learn_step += 1
        if self.learn_step % self.update_freq == 0:
            self.target_net.load_state_dict(self.q_net.state_dict())

        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

In [ ]:
from SimulatedNetwork import SimulatedNetwork
from State.StaticStateSimple import StaticStateSimple
from Reward.LinearReward import LinearReward

env = SimulatedNetwork(action_dim=4, state_dim=8, state_function=StaticStateSimple(), reward_object=LinearReward())

agent = DQNAgent(state_dim=8, action_dim=4)

for episode in range(200):
    s, _ = env.reset()
    total_reward = 0
    for step in range(100):
        a = agent.select_action(s)
        s_, r, terminated, truncated, info = env.step(a)
        agent.store(s, a, r, s_, terminated)
        agent.learn()
        s = s_
        total_reward += r
        if terminated:
            break
    print(f"Episode {episode}, Reward: {total_reward:.2f}, Epsilon: {agent.epsilon:.3f}")